# GPTQ opt-6.7b

In [1]:
import os

# os.environ['http_proxy'] = ''
# os.environ['https_proxy'] = ''
os.environ['HF_HOME'] = '/root/autodl-tmp/models'
os.environ['HF_HUB_CACHE'] = '/root/autodl-tmp/models/hub'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# os.environ['KMP_AFFINITY'] = 'disabled'

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_name_or_path = "facebook/opt-6.7b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="wikitext2",
     desc_act=False,
)

按照[issue249](https://github.com/AutoGPTQ/AutoGPTQ/issues/249)的说明，重新从源码安装AutoGPTQ，解决`CUDA extension not installed`的问题

In [3]:
quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/transformers/modeling_utils.py:4674: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [4]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('qweight',
               tensor([[-1533367701,  2022282410, -1182414405,  ..., -1537760937,
                          968190105, -1218869078],
                       [-1973762678, -1990888091,  1451669112,  ...,  1767327094,
                         1485273242, -1769109111],
                       [ -890521657,  1705355194,  2042256023,  ...,  1401453177,
                         -963081656, -1212573545],
                       ...,
                       [-1192544404,   697191045,  1432856694,  ...,  1967820506,
                        -1482119368, -1787262823],
                       [-1736931943, -1753576812,  2027985786,  ..., -1757504693,
                         2090308806, -1987483739],
                       [ 1549191317,  1151064006, -1735993498,  ..., -1317428394,
                        -1182375288, -1199925157]], device='cuda:0', dtype=torch.int32)),
              ('qzeros',
               tensor(

In [5]:
# 保存模型权重
quant_model.save_pretrained("models/opt-6.7b-gptq")

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to see you're still around.
I'm still around, just not as much as I used to be.
